# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [115]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [116]:
data = "cities_data.csv"
weather_data = pd.read_csv(data)
weather_data = weather_data.dropna()
del weather_data["Unnamed: 0"]
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Castro,-24.7911,-50.0119,51.82,91,0,5.39
1,Mataura,-46.1927,168.8643,72.39,57,97,0.81
2,Punta Arenas,-53.1500,-70.9167,59.00,44,90,4.61
3,Thompson,55.7435,-97.8558,-25.60,62,90,14.97
4,Guerrero Negro,27.9769,-114.0611,59.54,77,0,8.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [117]:
gmaps.configure(api_key=g_key)

In [118]:
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"].astype(float)

In [119]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
dissipating=False, max_intensity=100,point_radius=3)
fig.add_layer(heat_layer)
plt.savefig('heatmaps.png')
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [120]:
vacation_cloud = weather_data[weather_data["Cloudiness"] <5]
vacation_min_temp = vacation_cloud[vacation_cloud["Max Temp"] > 65]
vacation_max_temp = vacation_min_temp[vacation_min_temp["Max Temp"] < 90]
vacation_wind = vacation_max_temp[vacation_max_temp["Wind Speed"] < 5]
vacation_humidity = vacation_wind[vacation_wind["Humidity"] < 70]

In [121]:
perfect_vacation = vacation_humidity.reset_index()
perfect_vacation.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,216,Otjimbingwe,-22.3500,16.1333,79.74,36,3,4.07
1,275,Tālcher,20.9500,85.2167,83.07,21,0,2.68
2,320,Dalbandin,28.8947,64.4101,70.12,18,3,2.82
3,407,Makakilo City,21.3469,-158.0858,69.48,68,1,3.51
4,417,Agartala,23.8364,91.2750,75.20,57,0,3.44


In [122]:
del perfect_vacation["index"]

In [123]:
perfect_vacation

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Otjimbingwe,-22.3500,16.1333,79.74,36,3,4.07
1,Tālcher,20.9500,85.2167,83.07,21,0,2.68
2,Dalbandin,28.8947,64.4101,70.12,18,3,2.82
3,Makakilo City,21.3469,-158.0858,69.48,68,1,3.51
4,Agartala,23.8364,91.2750,75.20,57,0,3.44
5,Iquique,-20.2208,-70.1431,69.80,60,0,1.14
6,Korhogo,9.4500,-5.6333,70.79,28,1,1.52


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [124]:
hotels = []
for city in range(len(perfect_vacation["City"])):
    lat = perfect_vacation.loc[city]["Lat"]
    lng = perfect_vacation.loc[city]["Lng"]
    city_loc = f"{lat},{lng}"
    params = {"location": city_loc, 
            "types": "hotels",
            "radius": 5000,
            "key": g_key}
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("no hotels found")
perfect_vacation["Nearest Hotel"] = hotels
perfect_vacation

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Nearest Hotel
0,Otjimbingwe,-22.3500,16.1333,79.74,36,3,4.07,Otjimbingwe
1,Tālcher,20.9500,85.2167,83.07,21,0,2.68,Talcher Town
2,Dalbandin,28.8947,64.4101,70.12,18,3,2.82,Dalbandin
3,Makakilo City,21.3469,-158.0858,69.48,68,1,3.51,Kapolei
4,Agartala,23.8364,91.2750,75.20,57,0,3.44,Agartala
5,Iquique,-20.2208,-70.1431,69.80,60,0,1.14,Iquique
6,Korhogo,9.4500,-5.6333,70.79,28,1,1.52,Korhogo


In [125]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perfect_vacation.iterrows()]
locations = perfect_vacation[["Lat", "Lng"]]

In [126]:
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
fig.add_layer(markers)
plt.savefig('heatmaps.png')
fig




Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>